Here, we will investigate how good a score can be achieved using character 10-grams.

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import sys
import random
import math
import gc
from scipy import stats
from nltk import FreqDist, ngrams, sent_tokenize, word_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.linear_model import SGDClassifier
from nltk.stem.porter import PorterStemmer
from scipy import sparse
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold, ParameterGrid, RandomizedSearchCV, train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import _pickle as pickle

In [2]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        p = pickle.Pickler(output) 
        p.fast = True 
        p.dump(obj)
        
def load_object(filename):
    with open(filename, 'rb') as f:
        x = pickle.load(f)
    return(x)

def sum_keys(d):
    return (0 if not isinstance(d, dict) else len(d) + sum(sum_keys(v) for v in d.values()))

# Load the data & tree construction

Constructing a dictionary of all 10-grams is intractable on an 8GB RAM computer. To make things feasible we have to trim the tree quite heavily. Imposing a lower limit in the count of grams still yields an intractable tree (some 20 million unique keys). Hence, we write a function to trim the tree. We do this based on a few ideas, including (i) imposing a stricter lower limit on k-grams with low k, (ii) selecting words that distinguish native from non-native using odds ratios and (iii) removing branches of the tree that add no information. 

Under (iii) we understand branches that stem from a unique internal node. E.g. "cyclop" corresponds almost uniquely to "encyclopedia" and adding n-grams such as "ncyclop" doesn't add further information to the tree. We take care of this trimming in a second function `construct_log_odds_dict`.

In [3]:
def char_distribution(m, lowerfreqlimit, training, LANGUAGES):
    """Calculate the char m grams distribution.
    @m: consider k-grams up to and including m for characters.
    @lowerfreqlimit: number below which we consider grams near-unique and irrelevant
    @training: training data to retrieve the language distribution from.
    @LANGUAGES: languages based on which we classify. Either native languages or "non-native"/"native" divide is possible.
    """
    
    char_dist = {}

    for language in LANGUAGES:
        char_dist[language] = dict(zip(range(1, m+1), [FreqDist() for i in range(1, m+1)]))
    
    for k in range(1, m+1):
        for language, text in training.itertuples(index=False):
            for sentence in sent_tokenize(text):
                
                # Note, for any gram, there exist 2 subgrams of all but the first and all of the last element. Let us
                # only update the dictionary if the total count of these subgrams exceeds the lower limit. This prevents
                # an unnecessary combinatorial explosion.                
                
                for gram in ngrams(sentence,k):
                    if k == 1: 
                        char_dist[language][k][gram] += 1
                    elif char_dist[language][k-1].get(gram[1:],0)+char_dist[language][k-1].get(gram[:-1],0) > 2*lowerfreqlimit:
                        char_dist[language][k][gram] += 1
                        
        print("Completed counting all {}-grams".format(k))
                                               
    return char_dist

def construct_lodds_ratio_dict(fname, alpha, lbo, lbc, fc):
    """ 
    In this function, we want to compute the odds ratio for each of the n-grams, and return a dictionary with these values.
    For each non-English language, we will add a pseudocount of .5 to prevent divisions by 0. We return an approximate lower
    bound at the alpha confidence level.
    @fname: File to load the language distribution from
    @alpha: Alpha on the log-odds ratio between we will not 
    @lbo: lower bound for lb on log odds ratio. Upperbound is its inverse (assume symmetry).
    @lbc: lower bound on the count. We want to be less harsh for higher k, so we select on lbc/k
    @fc: factor how much the children of a gram may account for them for the node to be superfluous.
         Note: unique k-grams have two children. Thus, factor 2 is expected for unique grams. Reasonable
         values thus order 2-4.
    """
    
    lang_dis = load_object(fname)
    z = stats.norm.ppf(1-alpha)
    m = len(lang_dis['EN'])
    
    disc = 0
    for lang in lang_dis.keys():
        if lang == 'EN':
            continue       
    
        # First let us prune the tree, i.e. remove any superfluous branches.
        for k in sorted(list(lang_dis[lang].keys()), reverse=True):
            if k == 1:
                continue
                                            
            # Prune the tree by removing elements that are superfluous. Let us define them as superfluous
            # if they represent 95% of observations of the parent node.
            for key in list(lang_dis[lang][k].keys()):
                                
                # Remove the key and get the count. If superfluous, delete but add to remainder.
                a = lang_dis[lang][k].pop(key,0)
                                
                if lang_dis[lang][k-1].get(key[1:],0) + lang_dis[lang][k-1].get(key[:-1],0) < fc*a:
                    disc+=1
                    lang_dis[lang][k]["REMAINDER"] += a
                else:
                    lang_dis[lang][k][key] = a
                    
        # Now prune the tree further with log-odds ratios.
        for k in lang_dis[lang].keys():
            
            b = sum(lang_dis[lang][k].values()) + .5   #Total grams in foreign language
            d = sum(lang_dis['EN'][k].values()) + .5   #Total grams in English
            rem = lang_dis[lang][k].pop("REMAINDER",0)
            
            for key in list(lang_dis[lang][k].keys()):
                    
                # Obtain the value by pop, i.e. delete key from dictionary.
                a = lang_dis[lang][k].pop(key,0) +.5   #Gram count for particular gram in foreign language
                c = lang_dis['EN'][k].get(key,0) +.5   #Gram count for particular gram in English
                
                # Log odds ratio
                log_odds_ratio = math.log((a/b)/(c/d))
                alpha_lim = z*math.sqrt(1/a+1/b+1/c+1/d)

                # Lower bound must be greater than limit, upper bound lower than lower limit.
                if log_odds_ratio - alpha_lim > math.log(lbo) or log_odds_ratio - alpha_lim < math.log(1/lbo):
                    if a > lbc/k:
                        lang_dis[lang][k][key] = a - .5
                    else:
                        lang_dis[lang][k]["REMAINDER"] += a
                else:
                    lang_dis[lang][k]["REMAINDER"] += a
                    
    print("Discarded {} keys that were superfluous".format(disc))
            
    # Remove English from the language dictionary.
    lang_dis["EN"].clear()

    return(lang_dis)

In [4]:
# Load same data as in other notebooks.
df = pd.read_csv("python_data/train",sep="\t",error_bad_lines=False,encoding="utf-8")
df = df.sample(frac=1, random_state = 54021)
df['native'] = np.where(df['native_lang']=='EN', "native", "non-native")

# Select training samples.
print("Loading the training and validation data...")
training = pd.concat([df[df.native == "non-native"].sample(sum(df.native == "native"), random_state = 1810), df[df.native=="native"]])
training = training.sample(frac=1, random_state = 1318910)
training.native = training.native.astype('category')
df = df.iloc[0:0]

Loading the training and validation data...


### Selecting grams to prevent memory issues.

Selecting grams based on log-odds ratios and then using these same grams as features for learning from training data per se may lead to overfitting. Also, it is intractable to select the most important features on all training data. Hence, we first split training data into two sets. From the first set, we will construct a log-odds dictionary and select interesting grams. We will use this list on a second set selecting the most important features. This hopefully results in a robust set of grams based on which we can train.

In [5]:
# Split into two sets. First will be used to construct a log-odds dictionary, second to evaluate selected terms.
X_in_sample, X_out_sample = train_test_split(training[['native_lang','text_clean']], test_size=0.15, random_state=42)

In [6]:
# Create a character distribution from the training set. Set lowerlimit to 15, otherwise it won't save.
print("Training the character distribution")
char_dis = char_distribution(10, 15, X_in_sample, training.native_lang.unique())

# Clear all stuff in memory which is not the character distribution so that Pickling doesn't fail...
X_in_sample = X_in_sample.iloc[0:0]
X_out_sample = X_out_sample.iloc[0:0]
training = training.iloc[0:0]
gc.collect()

# Pickle the object.
save_object(char_dis,"trained_char_dis_10")
print("Character distribution constructed with {} unique grams".format(sum_keys(char_dis)))
char_dis.clear()

Training the character distribution
Completed counting all 1-grams
Completed counting all 2-grams


/home/hans/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: generator 'ngrams' raised StopIteration


Completed counting all 3-grams
Completed counting all 4-grams
Completed counting all 5-grams
Completed counting all 6-grams
Completed counting all 7-grams
Completed counting all 8-grams
Completed counting all 9-grams
Completed counting all 10-grams
Character distribution constructed with 22416133 unique grams


In [6]:
# Construct the log-odds dictionary. From this dictionary, retrieve the important grams.
lodds_ratio = construct_lodds_ratio_dict("trained_char_dis_10", 0.01, 11/10, 100, 4)
char_gram_list = []
for lang in lodds_ratio.keys():
    if lang == "EN":
        continue
    for k in lodds_ratio[lang].keys():
        for key in lodds_ratio[lang][k].keys():
            char_gram_list.append(key)
char_gram_list = set(char_gram_list)
char_gram_list = [''.join(gram) for gram in char_gram_list]
print("There are {} grams remaining which will be the basis for the initial vocabulary".format(len(char_gram_list)))

lodds_ratio.clear()
gc.collect()

Discarded 1069412 keys that were superfluous
There are 308177 grams remaining which will be the basis for the initial vocabulary


0

The list of character grams is still of an intractable size. Let us select important features from it with SelectFromModel. We need a sensible way to select features. Note that the features here essentially reduce to words/misspellings. Random forests seems to be a sensible way to select features. Let us do this.

In [7]:
random.seed(141921)
print("Constructing the out-of-sample document-term matrix")
char_vectorizer = CountVectorizer(ngram_range=(1,10), vocabulary = char_gram_list, analyzer="char", lowercase=False, dtype=np.float64)
char_vector = char_vectorizer.transform(X_out_sample.text_clean)
print("Training with out-of-sample document-term matrix")
X_out_sample['native'] = np.where(X_out_sample['native_lang']=='EN', "native", "non-native")
rf = RandomForestClassifier(n_estimators=500, min_samples_leaf = 10, n_jobs = -1)
rf.fit(char_vector, X_out_sample.native)

Constructing the out-of-sample document-term matrix
Training with out-of-sample document-term matrix


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
filtered_char_list = []
for gram, boolean in zip(char_gram_list, SelectFromModel(rf, threshold=1e-6, prefit = True).get_support()):
    if boolean:
        filtered_char_list.append(gram)
print("Length of the filtered list is {}".format(len(filtered_char_list)))

save_object(filtered_char_list, "selected_subset")

Length of the filtered list is 41790


### Restart python & reload the selected features. We use these to construct the DTM for validation and training data.

In [5]:
# Select training samples.
filtered_char_list = load_object("selected_subset")

print("Constructing the document-term matrix for 60% of training data")
training = training.sample(frac=0.6, random_state=45)
char_vectorizer = CountVectorizer(ngram_range=(1,10), vocabulary = filtered_char_list, analyzer="char", lowercase=False)
char_vector = char_vectorizer.transform(training.text_clean)
training.to_csv("python_data/training_60%_sample")
training_label = training.native
training = training.iloc[0:0]

# Load validation data.
validation = pd.read_csv("python_data/development",sep="\t",error_bad_lines=False,encoding="utf-8")
validation['native'] = np.where(validation['native_lang']=='EN', "native", "non-native")
validation = pd.concat([validation[validation.native == "non-native"].sample(sum(validation.native == "native"), random_state = 1), validation[validation.native=="native"]])
validation.native = validation.native.astype('category')

# Construct char vector for validation data.
print("Constructing DTM for validation data")
validation_char_vector = char_vectorizer.transform(validation.text_clean)
validation_label = validation.native
validation = validation.iloc[0:0]
sparse.save_npz("char_10_vector",char_vector)
sparse.save_npz("val_char_10_vector",validation_char_vector)

Constructing the document-term matrix for 60% of training data
Constructing DTM for validation data


### Reload again. Now load the DTMs.

In [3]:
# Load data and grams.
training = pd.read_csv("python_data/training_60%_sample")
validation = pd.read_csv("python_data/validation_tfidf")
training_label = training.native
validation_label = validation.native
validation = validation.iloc[0:0]
training = training.iloc[0:0]
gc.collect()

# Load DTMs
training_grams = sparse.load_npz("char_10_vector.npz")
validation_grams = sparse.load_npz("val_char_10_vector.npz")

In [4]:
# Multinomial Naive Bayes
print("Training the Naive Bayes estimator")
clf = MultinomialNB(alpha=0.01).fit(training_grams, training_label)
predicted_NB = clf.predict(validation_grams)
accuracy = 1-sum(predicted_NB != validation_label)/len(predicted_NB)
print("Naive Bayes accuracy: {}".format(accuracy))
clf = SGDClassifier(loss="hinge",penalty="l2",alpha=1e-4, random_state=42, max_iter=500, tol=None)
clf.fit(training_grams, training_label)
predicted_SVM = clf.predict(validation_grams)
accuracy = 1-sum(predicted_SVM != validation_label)/len(predicted_SVM)
print("Classifying with SVM with gradient descent gives accuracy of {}%".format(accuracy*100))

Training the Naive Bayes estimator
Naive Bayes accuracy: 0.6472017161966055
Classifying with SVM with gradient descent gives accuracy of 68.17149346961952%
